In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

import pathlib
import random
import os

In [2]:
dataPath = '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/'
dataRoot = pathlib.Path(dataPath)

In [3]:
imageDirs = []
for item in dataRoot.iterdir():
    imageDirs.append(item)

dataRoot = []
for path in imageDirs:
    dataRoot.append(pathlib.Path(path))

In [4]:
labeledImages = []
label = 0
classes = sorted(os.walk(dataPath).__next__()[1])
# List each sub-directory (the classes)
for c in classes:
    c_dir = os.path.join(dataPath, c)
    walk = os.walk(c_dir).__next__()
    # Add each image to the training set
    for sample in walk[2]:
        # Only keeps jpeg images
        if sample.endswith('.png') or sample.endswith('.jpeg'):
            labeledImages.append( (label, os.path.join(c_dir, sample)) )
    label += 1

In [6]:
labeledTrain = []
labeledTest = []
i = 0
for image in labeledImages:
    if(i < .8*len(labeledImages)):
        labeledTrain.append(image)
        i += 1
    else:
        labeledTest.append(image)
trainPaths = [x[1] for x in labeledTrain]
trainLabels = [x[0] for x in labeledTrain]
testPaths = [x[1] for x in labeledTest]
testLabels = [x[0] for x in labeledTest]
testSize = len(testLabels)

In [7]:
#trainPaths = []
#trainLabels = []
#testPaths = []
#testLabels = []
#for i in range(len(imagePaths)):
#    if(i < .8*len(imagePaths)):
#        trainPaths.append(imagePaths[i])
#        trainLabels.append(labels[i])
#    else:
#        testPaths.append(imagePaths[i])
#        testLabels.append(labels[i])

def loadAndPreprocessImage(path):
  image = tf.read_file(path)
  image = tf.image.decode_png(image, channels=3)
  image = tf.image.resize_images(image, [128, 128])
  image = image / 127.5 - 1.0  # normalize to [-1,1] range
  return image

In [8]:
def prepDataForConvNet(imagePaths, labels, batch_size):
    # Convert to Tensor
    imagePaths = tf.convert_to_tensor(imagePaths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)
    # Build a TF Queue, shuffle data
    image, label = tf.train.slice_input_producer([imagePaths, labels],
                                                 shuffle=True)

    # Read images from disk
    image = tf.read_file(image)
    image = tf.image.decode_png(image, channels=CHANNELS)

    # Resize images to a common size
    image = tf.image.resize_images(image, [128, 128])
    
    

    # Normalize
    image = image * 1.0/127.5 - 1.0

    # Create batches
    X, Y = tf.train.batch([image, label], batch_size=batch_size,
                          capacity=batch_size * 8,
                          num_threads=4)

    return X, Y

In [9]:
learning_rate = 0.001
num_steps = 50
batchSize = 128
display_step = 1

# Network Parameters
dropout = 0.75 # Dropout, probability to keep units

# Image Parameters_
N_CLASSES = 2 # CHANGE HERE, total number of classes
IMG_HEIGHT = 128 # CHANGE HERE, the image height to be resized to
IMG_WIDTH = 128 # CHANGE HERE, the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale

In [10]:
trainImages, trainLabels = prepDataForConvNet(trainPaths, trainLabels, batchSize)
testImages, testLabels = prepDataForConvNet(testPaths, testLabels, 1)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To 

In [11]:
#trainPathDS = tf.data.Dataset.from_tensor_slices(trainPaths)
#trainImageDS = trainPathDS.map(loadAndPreprocessImage, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#trainLabelDS = tf.data.Dataset.from_tensor_slices(tf.cast(trainLabels, tf.int64))
#trainDS = tf.data.Dataset.zip((trainImageDS, trainLabelDS))

#testPathDS = tf.data.Dataset.from_tensor_slices(testPaths)
#testImageDS = testPathDS.map(loadAndPreprocessImage, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#testLabelDS = tf.data.Dataset.from_tensor_slices(tf.cast(testLabels, tf.int64))
#testDS = tf.data.Dataset.zip((testImageDS, testLabelDS))

In [12]:
# Create model
def conv_net(images, n_classes, dropout, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):

        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(images, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2
        conv1 = tf.layers.max_pooling2d(conv1, 5, 3)

        # Convolution Layer with 32 filters and a kernel size of 5
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2 
        conv2 = tf.layers.max_pooling2d(conv2, 5, 3)
        
        conv3 = tf.layers.conv2d(conv2, 128, 3)
        conv3 = tf.layers.max_pooling2d(conv3, 3, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv3)

        # Fully connected layer (in contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)
        # Because 'softmax_cross_entropy_with_logits' already apply softmax,
        # we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out

    return out

In [13]:
logits_train = conv_net(trainImages, N_CLASSES, dropout, reuse=False, is_training=True)
# Create another graph for testing that reuse the same weights
logits_test = conv_net(trainImages, N_CLASSES, dropout, reuse=True, is_training=False)


logits_test_test = conv_net(testImages, N_CLASSES, dropout, reuse=True, is_training=False)

# Define loss and optimizer (with train logits, for dropout to take effect)
loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits_train, labels=trainLabels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.cast(trainLabels, tf.int64))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

correct_pred_test = tf.equal(tf.argmax(logits_test_test, 1), tf.cast(testLabels, tf.int64))
accuracy_test = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Saver object
saver = tf.train.Saver()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Start the data queue
    tf.train.start_queue_runners()

    # Training cycle
    for step in range(1, num_steps+1):

        if step % display_step == 0:
            # Run optimization and calculate batch loss and accuracy
            _, loss, acc = sess.run([train_op, loss_op, accuracy])
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        else:
            # Only run the optimization op (backprop)
            sess.run(train_op)
            
    for step in range(1, testSize + 1):

        if step % display_step == 0:
            # Run optimization and calculate batch loss and accuracy
            acc2 = sess.run(accuracy_test)
            print("Step " + str(step) + ", Training Accuracy= " + str(acc2))

            

    print("Optimization Finished!")
    # Save your model
    saver.save(sess, 'my_tf_model4')

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Step 1, Minibatch Loss= 0.6914, Training Accuracy= 0.570
Step 2, Minibatch Loss= 0.9094, Training Accuracy= 0.641
Step 3, Minibatch Loss= 2.2898, Training Accuracy= 0.320
Step 4, Minibat

Step 79, Training Accuracy= 0.890625
Step 80, Training Accuracy= 0.875
Step 81, Training Accuracy= 0.9296875
Step 82, Training Accuracy= 0.875
Step 83, Training Accuracy= 0.890625
Step 84, Training Accuracy= 0.859375
Step 85, Training Accuracy= 0.890625
Step 86, Training Accuracy= 0.890625
Step 87, Training Accuracy= 0.9296875
Step 88, Training Accuracy= 0.8984375
Step 89, Training Accuracy= 0.890625
Step 90, Training Accuracy= 0.921875
Step 91, Training Accuracy= 0.90625
Step 92, Training Accuracy= 0.9140625
Step 93, Training Accuracy= 0.8671875
Step 94, Training Accuracy= 0.8984375
Step 95, Training Accuracy= 0.875
Step 96, Training Accuracy= 0.8515625
Step 97, Training Accuracy= 0.890625
Step 98, Training Accuracy= 0.9296875
Step 99, Training Accuracy= 0.890625
Step 100, Training Accuracy= 0.90625
Step 101, Training Accuracy= 0.9296875
Step 102, Training Accuracy= 0.9296875
Step 103, Training Accuracy= 0.8671875
Step 104, Training Accuracy= 0.921875
Step 105, Training Accuracy= 0.86

Step 296, Training Accuracy= 0.90625
Step 297, Training Accuracy= 0.921875
Step 298, Training Accuracy= 0.90625
Step 299, Training Accuracy= 0.9375
Step 300, Training Accuracy= 0.859375
Step 301, Training Accuracy= 0.8671875
Step 302, Training Accuracy= 0.9140625
Step 303, Training Accuracy= 0.9140625
Step 304, Training Accuracy= 0.9296875
Step 305, Training Accuracy= 0.859375
Step 306, Training Accuracy= 0.890625
Step 307, Training Accuracy= 0.90625
Step 308, Training Accuracy= 0.9375
Step 309, Training Accuracy= 0.921875
Step 310, Training Accuracy= 0.9140625
Step 311, Training Accuracy= 0.8984375
Step 312, Training Accuracy= 0.8828125
Step 313, Training Accuracy= 0.90625
Step 314, Training Accuracy= 0.890625
Step 315, Training Accuracy= 0.921875
Step 316, Training Accuracy= 0.9375
Step 317, Training Accuracy= 0.9140625
Step 318, Training Accuracy= 0.921875
Step 319, Training Accuracy= 0.921875
Step 320, Training Accuracy= 0.890625
Step 321, Training Accuracy= 0.8671875
Step 322, Tra

Step 513, Training Accuracy= 0.9296875
Step 514, Training Accuracy= 0.921875
Step 515, Training Accuracy= 0.9375
Step 516, Training Accuracy= 0.875
Step 517, Training Accuracy= 0.9453125
Step 518, Training Accuracy= 0.8984375
Step 519, Training Accuracy= 0.8828125
Step 520, Training Accuracy= 0.921875
Step 521, Training Accuracy= 0.921875
Step 522, Training Accuracy= 0.8359375
Step 523, Training Accuracy= 0.890625
Step 524, Training Accuracy= 0.890625
Step 525, Training Accuracy= 0.8828125
Step 526, Training Accuracy= 0.9140625
Step 527, Training Accuracy= 0.9140625
Step 528, Training Accuracy= 0.921875
Step 529, Training Accuracy= 0.875
Step 530, Training Accuracy= 0.859375
Step 531, Training Accuracy= 0.875
Step 532, Training Accuracy= 0.8828125
Step 533, Training Accuracy= 0.9609375
Step 534, Training Accuracy= 0.8203125
Step 535, Training Accuracy= 0.9296875
Step 536, Training Accuracy= 0.890625
Step 537, Training Accuracy= 0.8984375
Step 538, Training Accuracy= 0.9140625
Step 539,

Step 729, Training Accuracy= 0.9140625
Step 730, Training Accuracy= 0.90625
Step 731, Training Accuracy= 0.890625
Step 732, Training Accuracy= 0.90625
Step 733, Training Accuracy= 0.875
Step 734, Training Accuracy= 0.859375
Step 735, Training Accuracy= 0.84375
Step 736, Training Accuracy= 0.9609375
Step 737, Training Accuracy= 0.9140625
Step 738, Training Accuracy= 0.90625
Step 739, Training Accuracy= 0.890625
Step 740, Training Accuracy= 0.90625
Step 741, Training Accuracy= 0.890625
Step 742, Training Accuracy= 0.9140625
Step 743, Training Accuracy= 0.9375
Step 744, Training Accuracy= 0.90625
Step 745, Training Accuracy= 0.8671875
Step 746, Training Accuracy= 0.8984375
Step 747, Training Accuracy= 0.953125
Step 748, Training Accuracy= 0.875
Step 749, Training Accuracy= 0.890625
Step 750, Training Accuracy= 0.9140625
Step 751, Training Accuracy= 0.890625
Step 752, Training Accuracy= 0.921875
Step 753, Training Accuracy= 0.859375
Step 754, Training Accuracy= 0.8515625
Step 755, Training

...

Step 4947, Training Accuracy= 0.875
Step 4948, Training Accuracy= 0.875
Step 4949, Training Accuracy= 0.8828125
Step 4950, Training Accuracy= 0.90625
Step 4951, Training Accuracy= 0.8828125
Step 4952, Training Accuracy= 0.8671875
Step 4953, Training Accuracy= 0.8828125
Step 4954, Training Accuracy= 0.9453125
Step 4955, Training Accuracy= 0.8984375
Step 4956, Training Accuracy= 0.921875
Step 4957, Training Accuracy= 0.9453125
Step 4958, Training Accuracy= 0.9375
Step 4959, Training Accuracy= 0.921875
Step 4960, Training Accuracy= 0.953125
Step 4961, Training Accuracy= 0.9140625
Step 4962, Training Accuracy= 0.875
Step 4963, Training Accuracy= 0.890625
Step 4964, Training Accuracy= 0.90625
Step 4965, Training Accuracy= 0.9140625
Step 4966, Training Accuracy= 0.90625
Step 4967, Training Accuracy= 0.8984375
Step 4968, Training Accuracy= 0.9140625
Step 4969, Training Accuracy= 0.890625
Step 4970, Training Accuracy= 0.90625
Step 4971, Training Accuracy= 0.890625
Step 4972, Training Accuracy=

Step 5158, Training Accuracy= 0.921875
Step 5159, Training Accuracy= 0.90625
Step 5160, Training Accuracy= 0.8671875
Step 5161, Training Accuracy= 0.921875
Step 5162, Training Accuracy= 0.90625
Step 5163, Training Accuracy= 0.875
Step 5164, Training Accuracy= 0.875
Step 5165, Training Accuracy= 0.9609375
Step 5166, Training Accuracy= 0.921875
Step 5167, Training Accuracy= 0.921875
Step 5168, Training Accuracy= 0.90625
Step 5169, Training Accuracy= 0.953125
Step 5170, Training Accuracy= 0.9296875
Step 5171, Training Accuracy= 0.9375
Step 5172, Training Accuracy= 0.9375
Step 5173, Training Accuracy= 0.921875
Step 5174, Training Accuracy= 0.8984375
Step 5175, Training Accuracy= 0.8984375
Step 5176, Training Accuracy= 0.8984375
Step 5177, Training Accuracy= 0.890625
Step 5178, Training Accuracy= 0.9140625
Step 5179, Training Accuracy= 0.8984375
Step 5180, Training Accuracy= 0.96875
Step 5181, Training Accuracy= 0.8671875
Step 5182, Training Accuracy= 0.953125
Step 5183, Training Accuracy= 

Step 5368, Training Accuracy= 0.9140625
Step 5369, Training Accuracy= 0.9375
Step 5370, Training Accuracy= 0.875
Step 5371, Training Accuracy= 0.8984375
Step 5372, Training Accuracy= 0.890625
Step 5373, Training Accuracy= 0.9140625
Step 5374, Training Accuracy= 0.9296875
Step 5375, Training Accuracy= 0.8828125
Step 5376, Training Accuracy= 0.9375
Step 5377, Training Accuracy= 0.8671875
Step 5378, Training Accuracy= 0.921875
Step 5379, Training Accuracy= 0.8359375
Step 5380, Training Accuracy= 0.8828125
Step 5381, Training Accuracy= 0.8984375
Step 5382, Training Accuracy= 0.9140625
Step 5383, Training Accuracy= 0.921875
Step 5384, Training Accuracy= 0.8671875
Step 5385, Training Accuracy= 0.875
Step 5386, Training Accuracy= 0.9296875
Step 5387, Training Accuracy= 0.9140625
Step 5388, Training Accuracy= 0.921875
Step 5389, Training Accuracy= 0.84375
Step 5390, Training Accuracy= 0.9453125
Step 5391, Training Accuracy= 0.859375
Step 5392, Training Accuracy= 0.890625
Step 5393, Training Ac

Exception in thread QueueRunnerThread-batch_1/fifo_queue-batch_1/fifo_queue_enqueue:
Traceback (most recent call last):
  File "/usr/local/anaconda3-4.4.0/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda3-4.4.0/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/training/queue_runner_impl.py", line 257, in _run
    enqueue_callable()
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1257, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node batch_1/fifo_queue_enqueue}}]]
Exception in thread QueueRunn

ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[{{node batch_1/fifo_queue_enqueue}}]]


In [14]:
#LEFT OFF AT PIPE THE DATASET TO A MODEL
#NEED TO:
#    SEPERATE DATA INTO TRAINING AND TESTING ###DONE
#    TRAIN MODEL
#    TEST MODEL

ERROR:tensorflow:Exception in QueueRunner: Dequeue operation was cancelled
	 [[{{node input_producer/input_producer_Dequeue}}]]


Exception in thread QueueRunnerThread-batch_1/fifo_queue-batch_1/fifo_queue_enqueue:
Traceback (most recent call last):
  File "/usr/local/anaconda3-4.4.0/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda3-4.4.0/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/training/queue_runner_impl.py", line 257, in _run
    enqueue_callable()
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1257, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node batch_1/fifo_queue_enqueue}}]]

Exception in thread QueueRun

ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[{{node batch/fifo_queue_enqueue}}]]


Exception in thread QueueRunnerThread-batch/fifo_queue-batch/fifo_queue_enqueue:
Traceback (most recent call last):
  File "/usr/local/anaconda3-4.4.0/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda3-4.4.0/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/training/queue_runner_impl.py", line 257, in _run
    enqueue_callable()
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1257, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Dequeue operation was cancelled
	 [[{{node input_producer/input_producer_Dequeue}}]]

Exception in thread Q